# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# current working directory
print(os.getcwd())


filepath = os.getcwd() + '/event_data'

#list of files and collect filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
query = "DROP TABLE IF EXISTS music_history"
try:
    session.execute(query)
except Exception as e:
    print(e)   

query = "DROP TABLE IF EXISTS song_history"
try:
    session.execute(query)
except Exception as e:
    print(e)   

query = "DROP TABLE IF EXISTS user_history"
try:
    session.execute(query)
except Exception as e:
    print(e)       
    
#session_id and item_in_session ARE PRIMARY KEYS TO MAKE RECORDS UNIQUE    
query = "CREATE TABLE IF NOT EXISTS music_history "
query = query + "(session_id int,item_in_session int, artist_name text, song_title text,song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)      
    
#user_id and session_id ARE PRIMARY KEYS, IN ORDER TO BE UNIQUE    
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(user_id int, session_id int,item_in_session int, artist_name text, song_title text,  first_name text, last_name text,  PRIMARY KEY ((user_id, session_id),item_in_session)"
query = query + ") with clustering order by (item_in_session asc)"
try:
    session.execute(query)
except Exception as e:
    print(e)     
    
#SONG_HISTORY PARTITIONED BY SONG_TITLE and ARTIST_NAME and USER_ID, BECAUSE THE TITLE CAN REPEAT AND WE NEED TO LIST MORE THAN ONE USER
query = "CREATE TABLE IF NOT EXISTS song_history "
query = query + "(song_title text, artist_name text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title,artist_name,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)     
        

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
                
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        prepared = """INSERT INTO music_history (session_id,item_in_session,artist_name, song_title,song_length) VALUES (%s,%s,%s,%s,%s)"""
        session.execute(prepared,(int(sessionId), int(itemInSession),artist_name,song,float(length),))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        prepared = session.prepare("""INSERT INTO song_history (song_title,artist_name,user_id, first_name , last_name ) VALUES (?,?,?,?,?)""")
        session.execute(prepared, (song, artist_name, int(userId) , user_name,user_last_name))    


In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        prepared = """INSERT INTO user_history(user_id, session_id, item_in_session, artist_name, song_title, first_name , last_name ) VALUES (%s,%s,%s,%s,%s,%s,%s)"""
        session.execute(prepared, (int(userId), int(sessionId), int(itemInSession), artist_name, song,  user_name , user_last_name))

In [15]:
query = "select artist_name,song_title,song_length,session_id,item_in_session from music_history WHERE session_id = 338 AND item_in_session= 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length, row.session_id, row.item_in_session)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
query = "select artist_name,song_title,first_name , last_name from user_history WHERE user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name) 
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [18]:
query = "select song_title,artist_name, user_id ,first_name,last_name from song_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title,row.artist_name, row.user_id, row.first_name, row.last_name) 

All Hands Against His Own The Black Keys 29 Jacqueline Lynch
All Hands Against His Own The Black Keys 80 Tegan Levine
All Hands Against His Own The Black Keys 95 Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "DROP TABLE IF EXISTS music_history"
try:
    session.execute(query)
except Exception as e:
    print(e)   

query = "DROP TABLE IF EXISTS song_history"
try:
    session.execute(query)
except Exception as e:
    print(e)   

query = "DROP TABLE IF EXISTS user_history"
try:
    session.execute(query)
except Exception as e:
    print(e)     

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()